# Import basic libraries

Let's import some basic stuff

## TODO
- Add the nodes in the CityJSON file, keeping the relationships between nodes and edges (linestrings). The nodes have semantics and atttributes.
- Find a small area and compute widths.
- Create LoD0.1 with single lines and attributes for two-way and same for LoD0.2.

# Ideas for modelling

1. Every city object contains `MultiPoints` (nodes of LoD0.1), `MultiLineStrings` (edges of LoD0.1) and `MultiSurfaces `(for LoD1+).
2. Every city object contains `MultiLineStrings` (geometry of edges of LoD0.1) and `MultiSurfaces` (for LoD1+). The network topology (nodes and edges) is defined in its own `"+network"` portion of the city model.
3. Every city object has its own `MultiSurfaces` (for LoD1+). Then the actual network (nodes, edges and their geometry) is stored in `"+network"`.
4. Every node is its own city object. Every edge is its own city object. Every surface is its own city object. Then `CityObjectGroups` are used to relate them.

### Fix for `fiona`

*`fiona` has an issue with GDAL 3.0. Better set your `GDAL_DATA` path to fiona's installation (contains GDAL 2.4.4) prior to running this script.*

In [1]:
import geopandas
import pandas as pd
import osmnx as ox
import networkx as nx
import shapely

# Download the OSM data

This will load the osm file to a network

In [2]:
roads_file = '../data/Den Haag/BGT_Roads.gpkg'

bgt_roads = geopandas.read_file(roads_file)
bgt_roads.crs = "EPSG:28992"

bbox = bgt_roads.to_crs("EPSG:4326").total_bounds

Load the OSM data:

In [3]:
# # We don't want edges to be loaded twice
ox.config(all_oneway=True, useful_tags_way=['bridge', 'tunnel', 'oneway', 'lanes', 'ref', 'name', 'highway', 'maxspeed', 'service', 'access', 'area', 'landuse', 'width', 'est_width', 'junction', 'layer'])

network_graph = ox.graph_from_bbox(bbox[3], bbox[1], bbox[2], bbox[0], simplify=False)
# We simplify the graph afterwards, so that osmids are not "smashed" together
network_graph = ox.simplify_graph(network_graph, strict=False)

We create `UUID` for every edge:

In [5]:
import uuid
import hashlib
import json
from cityhash import CityHash32

hashes = []
for u,v,a in network_graph.edges(data=True):
    temp = {
        "osmid": a['osmid'],
        "length": a['length']
    }
    a['uuid'] = CityHash32(uuid.uuid4().hex)
    hashes.append(a['uuid'])

# Ensure no conflict between the UUIDs
assert len(hashes) == len(set(hashes))

Then we export the edges to a GeoDataFrame:

In [6]:
nodes, edges = ox.graph_to_gdfs(network_graph)

edges = edges.to_crs("EPSG:28992")

len(edges)

54841

In [258]:
nodes, edges = ox.graph_to_gdfs(network_graph)

edges.head()

,osmid,name,highway,length,uuid,maxspeed,geometry,bridge,oneway,layer,...,ref,access,service,junction,tunnel,width,area,u,v,key
0,120925790,Kruisweg,cycleway,8.834,398641665,NaN,"LINESTRING (4.40510 52.04530, 4.40520 52.04525)",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1631846404,1354677828,0
1,7448225,De Poort,unclassified,88.251,3326118792,50,"LINESTRING (4.40510 52.04530, 4.40515 52.04533...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1631846404,3505997128,0
2,150260819,NaN,cycleway,13.757,377892979,NaN,"LINESTRING (4.40494 52.04534, 4.40479 52.04542)",yes,yes,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1631846411,1631846419,0
3,150260822,NaN,cycleway,13.285,1082770538,NaN,"LINESTRING (4.40469 52.04535, 4.40472 52.04533...",yes,yes,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1631846416,1631846391,0
4,150260821,NaN,cycleway,7.731,794515698,NaN,"LINESTRING (4.40479 52.04542, 4.40468 52.04542)",NaN,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1631846419,45004484,0


In [7]:
nodes.to_file('output/nodes.geojson', driver='GeoJSON')

## Filter drive and service roads only

Filter roads to only motorways:

In [8]:
highway_types = [
    "primary",
    "secondary",
    "motorway",
    "trunk",
    "tertiary",
    "unclassified",
    "residential",

    "motorway_link",
    "trunk_link",
    "primary_link",
    "secondary_link",
    "tertiary_link",

    "living_street",
    "service",
    "pedestrian",
#     "track",
    "bus_guideway",
    "escape",
    "raceway",
    "road"
]

road_edges = edges[(edges['highway'].isin(highway_types))]

road_edges.to_file('output/road_edges.geojson', driver='GeoJSON')
len(road_edges)

28434

# Load the BGT data

In [9]:
bgt_roads = geopandas.read_file(roads_file)

bgt_roads.crs = "EPSG:28992"

len(bgt_roads)

117366

## Compute primary-secondary road polygons

We need to assign secondary road polygons (parking spaces, sidewalks) to the primary roads

### Filter primary roads

Primary types are those that vehicles are supposed to move:

In [10]:
bgt_road_types = [
    'rijbaan lokale weg',
    'rijbaan regionale weg',
    'rijbaan autosnelweg',
    'rijbaan autoweg'
    # 'overweg' # Maybe?
]

bgt_roads = bgt_roads[bgt_roads['eindRegistratie'].isnull()]

bgt_roads['parent_gml_id'] = bgt_roads['gml_id']

bgt_main_roads = bgt_roads[bgt_roads['function'].isin(bgt_road_types)]

len(bgt_main_roads)

11996

### Identify parent for every secondary road

Find the secondary roads that touch the main ones:

In [11]:
bgt_secondary_roads = bgt_roads[~bgt_roads['function'].isin(bgt_road_types)]

len(bgt_secondary_roads)

42655

Let's overlay the secondary roads with the main roads. The result should be (mostly) linestrings accross their common boundaries:

In [12]:
bgt_common_boundaries = geopandas.overlay(bgt_secondary_roads, bgt_main_roads, how='intersection', keep_geom_type=False)

bgt_common_boundaries.crs = "EPSG:28992"

len(bgt_common_boundaries)

55805

We need to compute the longest common boundary per `gml_id` to pick the respective parent `gml_id`:

In [13]:
bgt_common_boundaries['length'] = bgt_common_boundaries['geometry'].length

max_rows = bgt_common_boundaries[['gml_id_1', 'function_1', 'gml_id_2', 'geometry', 'length']].groupby('gml_id_1')['length'].idxmax()

In [14]:
import numpy as np

max_rows = [a for a in max_rows if not np.isnan(a)]

Now, let's join this with the original secondary rows table to bring back their original geometries:

In [15]:
temp_roads = bgt_common_boundaries.loc[max_rows]
temp_roads = temp_roads[['gml_id_1', 'function_1', 'gml_id_2', 'geometry']]
temp_roads = temp_roads.rename(columns={'gml_id_1': 'gml_id', 'function_1': 'function', 'gml_id_2': 'parent_gml_id'})
temp_roads = temp_roads.set_index('gml_id')

bgt_roads_parented = temp_roads.merge(bgt_secondary_roads.set_index('gml_id'), on='gml_id', suffixes=('', '_right'))
bgt_roads_parented = bgt_roads_parented[['function', 'parent_gml_id', 'geometry_right']].rename(columns={'geometry_right': 'geometry'})
bgt_roads_parented = bgt_roads_parented.reset_index()

bgt_roads_parented = geopandas.GeoDataFrame(bgt_roads_parented, crs="EPSG:28992")

len(bgt_roads_parented)

31472

### Merge primary and secondary roads

Concatenate everything:

In [16]:
bgt_all_roads = pd.concat([bgt_roads_parented, bgt_main_roads[['gml_id', 'function', 'parent_gml_id', 'geometry']]])

bgt_all_roads.to_file('output/bgt_all_roads.geojson', driver='GeoJSON')

len(bgt_all_roads)

43468

# Intersect roads network with polygon
First, we intersect the road lines with the BGT polygons (to create nodes at the polygon boundaries):

In [17]:
roads = geopandas.overlay(road_edges, bgt_all_roads, how='intersection')

Export the intersected roads to `GeoJSON`:

In [18]:
roads.to_file('output/roads.json', driver='GeoJSON')

roads.head()

,osmid,name,highway,length,uuid,maxspeed,bridge,oneway,layer,lanes,...,tunnel,width,area,u,v,key,gml_id,function,parent_gml_id,geometry
0,7448225,De Poort,unclassified,88.251,3326118792,50,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1631846404,3505997128,0,b4480b307-9d3a-4ec5-bd72-9193624573c3,fietspad,b89af05c2-e396-4adb-9ccd-56c255932cf8,"LINESTRING (87626.736 451231.456, 87627.243 45..."
1,7448225,De Poort,unclassified,13.738,3335201683,50,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,45003347,1631846404,0,b4480b307-9d3a-4ec5-bd72-9193624573c3,fietspad,b89af05c2-e396-4adb-9ccd-56c255932cf8,"LINESTRING (87625.235 451230.227, 87626.736 45..."
2,7448225,De Poort,unclassified,88.251,3326118792,50,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1631846404,3505997128,0,b5486f8df-7f30-4f2c-8123-625d325a23e2,fietspad,ba320a171-755e-4894-b0ad-c75c2300db96,"LINESTRING (87682.067 451286.203, 87683.972 45..."
3,7448225,De Poort,unclassified,88.251,3326118792,50,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1631846404,3505997128,0,ba320a171-755e-4894-b0ad-c75c2300db96,rijbaan lokale weg,ba320a171-755e-4894-b0ad-c75c2300db96,"MULTILINESTRING ((87627.243 451231.947, 87629...."
4,343772191,NaN,service,9.554,3286716675,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3505997128,3505997129,0,ba320a171-755e-4894-b0ad-c75c2300db96,rijbaan lokale weg,ba320a171-755e-4894-b0ad-c75c2300db96,"LINESTRING (87689.518 451293.687, 87692.336 45..."


# Export to CityJSON

Let's export everything to CityJSON.

First, we'll define how the established intersected lines will be translated to `Road` objects:

In [19]:
def process_ring(line, vertices):
    """Returns the boundaries array of a single linear rings.
    
    Arguments:
    line -- The LineString to be processed (represents the ring)
    vertices -- the global vertices list of the CityJSON
    """
    points = [[x, y, 0] for x, y in list(line.coords)]
    indices = [i + len(vertices) for i in range(len(points))]
    for p in points:
        vertices.append(p)

    return indices

def process_linestring(geom, vertices):
    """Returns the boundaries array of a linear element.
    
    This can process LineString or MultiLineString geometries.
    
    Arguments:
    geom -- the (linear) geometry to be processed
    vertices -- the global vertices list of the CityJSON
    """
    if geom.type == "LineString":
        indices = [process_ring(geom, vertices)]
    else:
        indices = []
        
        for l in geom.geoms:
            indices.append(process_ring(l, vertices))
        
    return indices

def create_geometry(geom, lod, vertices):
    """Returns a CityJSON geometry from a single shapely geometry.
    
    This will also append the 'vertices' list with new vertices.
    
    Arguments:
    geom -- the shapely geometry to be processed
    lod -- the lod of the resulting CityJSON geometry
    vertices -- the global vertices list of the CityJSON
    """
    indices = []
    
    if geom.type == "LineString" or geom.type == "MultiLineString":
        geom_type = "MultiLineString"
        
        indices = process_linestring(geom, vertices)
    elif geom.type == "Polygon":
        geom_type = "MultiSurface"
        
        indices = [process_linestring(geom.boundary, vertices)]
    else:
        raise TypeError(geom.type)
    
    return {
        "type": geom_type,
        "lod": lod,
        "boundaries": indices
    }

def create_geom_with_semantics(map_func, features, lod, vertices, geom_column='geometry'):
    """Returns a CityJSON geometry with semantic elements from multiple features.
    
    Semantic elements are semantic lines (for LineString) or surfaces (for
    MultiSurface).
    
    Arguments:
    map_func -- function that maps an input feature to an output semantic element
    features -- list of features to be processed
    lod -- the lod of the resulting CityJSON geometry
    vertices -- the global vertices list of the CityJSON
    """
    boundaries = []
    semantics = {
        "surfaces": [],
        "values": []
    }
    
    # Get the resulting geometry type from the first feature
    geom = features[0][geom_column]
    if geom.type == "LineString" or geom.type == "MultiLineString":
        geom_type = "MultiLineString"
    elif geom.type == "Polygon":
        geom_type = "MultiSurface"
    else:
        raise TypeError(geom.type)
    
    # Process all features
    i = 0
    for f in features:
        surface = map_func(f)
        
        if geom_type == "MultiLineString":
            indices = process_linestring(f[geom_column], vertices)
            for l in indices:
                boundaries.append(l)
                semantics["surfaces"].append(surface)
                semantics["values"].append(i)
                i = i + 1
        else:
            indices = process_linestring(f[geom_column].boundary, vertices)
        
            boundaries.append(indices)
            semantics["surfaces"].append(surface)
            semantics["values"].append(i)
            i = i + 1
    
    return {
        "type": geom_type,
        "lod": lod,
        "boundaries": boundaries,
        "semantics": semantics
    }

def osm_semantics_map(feature):
    obj = {
        "type": feature['highway'],
        "osm_id": feature['osmid'],
        "oneway": feature['oneway'] if isinstance(feature['oneway'], str) else "no"
    }
    
    if isinstance(feature['name'], str):
        obj["name"] = feature['name']
    
    if isinstance(feature['maxspeed'], str):
        obj["maxspeed"] = feature['maxspeed']
    
    return obj

def bgt_semantics_map(feature):
    return {
        "type": feature['function'],
        "gml_id": feature['gml_id']
    }

def create_cityobject(feature, vertices):
    """Create a CityJSON city object from a single OSM linear features."""
    return {
        "type": "Road",
        "attributes": {
            "osm_id": feature['osmid'],
            "highway": feature['highway'],
        },
        "geometry": [
            create_geometry(feature['geometry'], "0.1", vertices)
        ]
    }

Now, let's run this against all intersected road segments:

In [20]:
vertices = []
objects = {}

road_groups = roads.groupby('parent_gml_id')

for gml_id, group in road_groups:
    features = [f for i, f in group.iterrows()]
    objects[gml_id] = {
        "type": "Road",
        "geometry": [
            create_geom_with_semantics(osm_semantics_map,
                                       features,
                                       '0.1',
                                       vertices)
        ]
    }

bgt_roads_groups = bgt_all_roads.groupby('parent_gml_id')

for main_gml_id, group in bgt_roads_groups:
    if main_gml_id in objects:
        features = [f for i, f in group.iterrows()]
        objects[main_gml_id]['geometry'].append(
            create_geom_with_semantics(bgt_semantics_map,
                                       features,
                                       '2',
                                       vertices)
        )

Finally, let's export everything as CityJSON:

In [21]:
import json
import io

output = {
  "type": "CityJSON",
  "version": "1.0",
  "CityObjects": objects,
  "vertices": vertices
}

with open('output/breda.json', 'w') as file:
    json.dump(output, file)

## Fix missing network and polygons
We should create individual road objects for the polygons (and respective network segments) that do not have a `parent_gml_id`.

# Calculate carriageways and lanes
- Compute the carriageways only from OSM attributes.
- Compute road width:
     - We exclude intersections.
- How we create the carriageways for twoway streets and more specifically for intersections.

NOTE: Case of a two-way road where we figure out that the road width doesn't fit two lanes (special case were two ways share the same carriageway).

In [22]:
from shapely.ops import unary_union
from shapely.geometry import Point, LineString, MultiLineString

def get_vertex(line, first=True):
    """Returns the first or last vertex of the line as a Point"""
    if line.type == "LineString":        
        return Point(line.coords[0 if first else -1])
    elif line.type == "MultiLineString" and len(line.geoms) == 1:
        return Point(line.geoms[0].coords[0 if first else -1])
    else:
        raise TypeError("This is MultiLineString with many parts!")

## Calculate width

### Using PostGIS

This is to calculate the widths in PostGIS using [this](https://github.com/willemhoffmans/bgt_wegbreedte/).

Let's import stuff and create the connection with the database:

In [23]:
# SO https://gis.stackexchange.com/a/239231

# Imports
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *
import pandas as pd
import geopandas as gpd

pg_user = "roads"
pg_pass = "roads"
pg_host = "localhost"
pg_port = 5432
pg_database = "road_widths"

# Creating SQLAlchemy's engine to use
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(pg_user, pg_pass, pg_host, pg_port, pg_database))

Now we export the BGT roads to the database with the appropriate names:

In [24]:
export_bgt = bgt_roads.copy()
export_bgt['geometrie_vlak'] = export_bgt['geometry'].apply(lambda x: WKTElement(x.wkt, srid=28992))

export_bgt.drop('geometry', 1, inplace=True)

export_bgt = export_bgt.rename(columns={'function':'bgt_functie'})

# Use 'dtype' to specify column's type
# For the geom column, we will use GeoAlchemy's type 'Geometry'
export_bgt.to_sql("wegdeel", engine, schema='bgt', if_exists='replace', index=False, 
                         dtype={'geometrie_vlak': Geometry('POLYGON', srid=28992)})

Let's create the required functions in the database:

In [25]:
import psycopg2

conn = psycopg2.connect(dbname=pg_database, user=pg_user, password=pg_pass)

file = open("wegbreedtes_script.sql", "r")
conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
cursor = conn.cursor()
cursor.execute(file.read())

Export the roads:

In [26]:
from shapely.geometry import MultiLineString

export_roads = road_edges.copy()
export_roads['geom'] = export_roads['geometry'].apply(lambda x: WKTElement(MultiLineString([x]).wkt, srid=28992))

export_roads.drop('geometry', 1, inplace=True)

# Use 'dtype' to specify column's type
# For the geom column, we will use GeoAlchemy's type 'Geometry'
export_roads.to_sql("road_edges", engine, if_exists='replace', index=False, 
                         dtype={'geom': Geometry('MULTILINESTRING', srid=28992)})

Now, run the function to compute the widths:

In [27]:
conn = psycopg2.connect(dbname=pg_database, user=pg_user, password=pg_pass)
conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)

cursor = conn.cursor()

cursor.execute("SELECT wh_wegbreedte_bgt_generiek ('road_edges', 'uuid', false, 0);")
for c in cursor:
    print(c)

('OK!',)


Load the resulting table with widths:

In [28]:
roads_with_width = geopandas.GeoDataFrame.from_postgis("SELECT * FROM breedte_analyse_nieuw.gemiddeldebreedte", conn, geom_col='geom' )

roads_with_width = roads_with_width.rename(columns={"geom": "geometry"})

roads_with_width.head()

,gid,typeweg,wegvakid,geometry,gem_breedte,dev_breedte,min_breedte,max_breedte,oper_breedte,afst_s,aantal,aantal_op
0,425,rijbaan lokale weg,140722843,"MULTILINESTRING ((82435.602 453252.408, 82408....",7.94,6.99,3.96,21.10,4.44,0.00,9,7
1,1228,fietspad,419670558,"MULTILINESTRING ((82421.047 454572.544, 82478....",1.26,0.04,1.21,1.35,1.26,0.26,8,6
2,1839,rijbaan lokale weg,636331810,"MULTILINESTRING ((88029.942 453627.941, 88002....",3.27,0.01,3.25,3.29,3.27,0.00,9,6
3,2092,OV-baan,726895795,"MULTILINESTRING ((82005.383 453557.591, 81972....",1.51,0.31,0.95,1.92,1.49,3.18,9,6
4,3638,OV-baan,1256576478,"MULTILINESTRING ((84127.551 454833.235, 84140....",2.43,0.00,2.43,2.43,2.43,1.38,9,9


# Create carriageways

We calculated the widths via PostGIS using [this](https://github.com/willemhoffmans/bgt_wegbreedte/) (embarassingly, the previous scripts didn't work).

Let's load the data with the width:

In [29]:
roads_width_final_final = roads_with_width.rename(columns={"wegvakid": "uuid"}).merge(road_edges.set_index("uuid"), on="uuid", suffixes=('', '_right'))

len(roads_width_final_final)

12711

Append the missing roads segments. We will set a road width of 0 for them. Also, we need to convert them to `MultiLineString` to match the rest of the geometries.

In [30]:
from shapely.geometry import MultiLineString
import numpy as np

def make_multilinestring(f):
    """Returns a geometry as MultiLineString if it is a LineString"""
    
    if f["geometry"].type == "LineString":
        return MultiLineString([f["geometry"]])
    
    return f["geometry"]

roads_width_final_final = pd.concat([roads_width_final_final, road_edges[road_edges['uuid'].isin(roads_width_final_final['uuid']) == False]], ignore_index=True)

roads_width_final_final.loc[roads_width_final_final[np.isnan(roads_width_final_final['oper_breedte'])].index, 'oper_breedte'] = 0

roads_width_final_final["geometry"] = roads_width_final_final.apply(make_multilinestring, axis=1)

len(roads_width_final_final)

28434

Cleanup duplicates:

In [31]:
def make_ccw(l):
    c = list(l.coords)
    if c[0][0] > c[-1][0]:
        c = c[::-1]
    l = LineString(c)
    
    return l

roads_width_final_final["wkt"] = roads_width_final_final["geometry"].apply(lambda l: make_ccw(l.geoms[0]).wkt)
roads_width_final_final = roads_width_final_final.drop_duplicates(subset=["wkt"])
roads_width_final_final = roads_width_final_final.drop(['wkt'], 1)

len(roads_width_final_final)

28431

Let's create the basic carriageways when `is_one_carriageway` is `false` (specific width and a two way street):

In [ ]:
from shapely.ops import linemerge

width_col = 'oper_breedte'

def find_neighbouring_edges(f, nodeid, remove_self=True):
    """Returns the road UUIDs of the neighbours of a feature at a given node"""
    
    uuids = [a['uuid'] for u, v, a in network_graph.in_edges(nodeid, data=True)] + [a['uuid'] for u, v, a in network_graph.out_edges(nodeid, data=True)]
    
    if remove_self:
        uuids.remove(f['uuid'])
    
    return roads_width_final_final[roads_width_final_final['uuid'].isin(uuids)]

def azimuth(point1, point2):
    '''azimuth between 2 shapely points (interval 0 - 360)'''
    angle = np.arctan2(point2.x - point1.x, point2.y - point1.y)
    return np.degrees(angle) if angle >= 0 else np.degrees(angle) + 360

def line_azimuth(line, beginning, min_length=0.001):
    """Return the azimuth of a LineString.
    
    Depending on the boolean provided, it will return the azimuth of the
    beginning (if True) or end (if False) of the LineString. In order to
    ensure stability, the algorithm expects that a min_length of segments
    is accounted for the two points tha will compute the azimuth.
    """
    
    if line.type == "MultiLineString":
        l = linemerge(line)
    else:
        l = line
    
    start_p = Point(l.coords[0 if beginning else -1])
    c_count = len(l.coords)
    for i in range(1, c_count):
        end_p = Point(l.coords[i if beginning else c_count - i - 1])
        tmp_line = LineString([start_p, end_p])
        if tmp_line.length > min_length:
            return azimuth(start_p, end_p)
    
    # If the min_length is too big, return the azimuth of the whole thing
    return azimuth(start_p, end_p)

def get_line_azimuth(line, reverse):
    """Returns the azimuth of a line based on its boundaries (end points).
    
    Better use the `line_azimuth` function instead!
    """
    if reverse:
        return azimuth(Point(line.coords[-1]), Point(line.coords[-2]))
    
    return azimuth(Point(line.coords[0]), Point(line.coords[1]))

def angle_of_features(f1, f2, nodeid, geom_column="geometry"):
    az = get_line_azimuth(f1[geom_column].geoms[0], f1['u'] != nodeid)
    n_az = get_line_azimuth(f2[geom_column].geoms[0], f2['u'] != nodeid)
        
    return abs(n_az - az)                
        

def is_one_carriageway(f, tolerance):
    return (f[width_col] < (6 - tolerance)) or (f[width_col] > (15 + tolerance))

def is_one_way(f):
    """Returns True if this feature is a one way"""
    
    if f['oneway'] == 'yes':
        return True
    
    if "junction" in f and f["junction"] == "roundabout":
        return True
    
    if "highway" in f and (f["highway"] == "motorway" or f["highway"] == "motorway_link"):
        return True
    
    return False

def get_continuation(f, neighbours, nodeid):
    if len(neighbours) == 1:
        return neighbours.iloc[0]
    
    for i, n in neighbours.reset_index().iterrows():
        theta = angle_of_features(f, n, nodeid)
        if theta > 160 and theta < 200:
            return n
    
    return None

def carriageway_placement(f):
    """Returns the width of the carriage ways"""
    if is_one_way(f):
        return 0
    
    u_neighbours = find_neighbouring_edges(f, f['u'])
    v_neighbours = find_neighbouring_edges(f, f['v'])
    
    if len(u_neighbours) == 1 and len(v_neighbours) == 1:
        un = u_neighbours.iloc[0]
        vn = v_neighbours.iloc[0]
        if un[width_col] > 6 and vn[width_col] > 6:
            return min(un[width_col], vn[width_col]) / 4
    
    if len(u_neighbours) > 1 and len(v_neighbours) > 1 and f["geometry"].length < 20:
        # Check the angle with some features
        cu = get_continuation(f, u_neighbours, f['u'])
        cv = get_continuation(f, v_neighbours, f['v'])
        if (not cu is None
            and not cv is None
            and (not is_one_carriageway(cu, 0)
                 or not is_one_carriageway(cv, 0))):
            return f[width_col] / 4
        else:
            return 0
    
    # Check if this is the last segment of a road which normally, if short, is
    # very much affected by the intersection and will end up with an unrealistically
    # high width. Therefore, we will choose its type by the previous segment of this
    # road.
    if f[width_col] > 15:
        c = get_continuation(f, u_neighbours, f['u'])
        if not c is None and not is_one_carriageway(c, 0):
            return c[width_col] / 4

        c = get_continuation(f, v_neighbours, f['v'])
        if not c is None and not is_one_carriageway(c, 0):
            return c[width_col] / 4

    if len(u_neighbours) > 1 and len(v_neighbours) == 1:
        n = v_neighbours.iloc[0]
        if n['geometry'].length > f['geometry'].length:
            if is_one_carriageway(n, 0):
                return 0
            else:
                return n[width_col] / 4
    if len(u_neighbours) == 1 and len(v_neighbours) > 1:
        n = u_neighbours.iloc[0]
        if n['geometry'].length > f['geometry'].length:
            if is_one_carriageway(n, 0):
                return 0
            else:
                return n[width_col] / 4

    # This is only the general case
    if is_one_carriageway(f, 0):
        return 0
    else:
        return f[width_col] / 4

def create_carriageway(f):
    geom = f['geometry']
    
    width = carriageway_placement(f)
    if width == 0:
        return geom
    
    # TODO: We check if it's an extreme and then we check if it's a dead-end
    #       in which case we just exclude or we return the original geometry
    
    return geom.geoms[0].parallel_offset(width, 'left').union(geom.geoms[0].parallel_offset(width, 'right'))

roads_width_final_final['carriage_ways'] = roads_width_final_final.apply(create_carriageway, axis=1)
geopandas.GeoDataFrame(roads_width_final_final[['uuid', 'u', 'v', 'carriage_ways']], geometry='carriage_ways', crs='EPSG:28992').to_file('output/ok.geojson', driver='GeoJSON')

print("Done")

# f = roads_width_final_final[roads_width_final_final['uuid'] == 3852453425].iloc[0]
# create_carriageway(f)

## Join carriageways

Now, we join the individual network segments with each other:
1. if both are single carriageway there is nothing to do
2. if both are dual carriageways then we compute the midpoint of the end and start node and we join them there
3. if one is single and another is dual then we join the dual to the end node of the single one (making a triangle).

We also identify continuation between carriageways in intersections and join them.

In [231]:
from shapely.ops import substring, linemerge
from shapely.geometry import LineString, MultiLineString
import numpy as np
import warnings

def is_first_vertex(f, u_nodeid):
    return f['u'] == u_nodeid

def cut_line(geom, dist, reverse=False):
    if reverse:
        return substring(geom, 0, geom.length - dist)
    else:
        return substring(geom, dist, geom.length)

def reverse_line(line):
    """Returns the line in the reverse orientation"""
    return LineString(reversed(line.coords))

def midpoint(pa, pb):
    return Point((pa.x + pb.x) / 2, (pa.y + pb.y) / 2)

def count_carriageways(f):
    if f['carriage_ways'].type == "LineString":
        return 1
    else:
        return len(f['carriage_ways'].geoms)

def as_line(geom):
    """Return a LineString from geom regardless if it's a MultiLineString or not"""

    if geom.type == "LineString":
        return geom
    else:
        return geom.geoms[0]

def merge_dual_to_single(geom, p, firstVertex):
    left_cw = geom.geoms[0] # Get left carriage way
    right_cw = geom.geoms[1] # Get right carriage way
    
    if left_cw.length < 1 or right_cw.length < 1:
        left_p = get_vertex(left_cw, not firstVertex) # Last point of left line
        right_p = get_vertex(right_cw, firstVertex) # First point of right line

        left_line = LineString([p, left_p])
        right_line = LineString([right_p, p])
    else:
        left_line = cut_line(left_cw, 1, not firstVertex)
        right_line = cut_line(right_cw, 1, firstVertex)

        left_p = get_vertex(left_line, firstVertex)
        right_p = get_vertex(right_line, not firstVertex)

        # Create the left and right side of the "triangle"
        left_side = LineString((p, left_p))
        right_side = LineString((p, right_p))

        # Merge the parts together
        left_line = linemerge(left_side.union(left_line))
        right_line = linemerge(right_side.union(right_line))

        if not get_vertex(left_line, firstVertex).equals(p):
            left_line = reverse_line(left_line)
        if not get_vertex(right_line, not firstVertex).equals(p):
            right_line = reverse_line(right_line)

    return MultiLineString([left_line, right_line])

def pick_closests_to(multi_p, p):
    """Gets the closest point of multi_p to p"""
    
    final_p = multi_p.geoms[0]
    d = p.distance(final_p)
    for pp in multi_p:
        if p.distance(pp) < d:
            final_p = pp
            d = p.distance(pp)
    
    return final_p

def join_dual_ways(geom, other_geom, isU, firstVertex):
    """Returns the end of a dual carriageway based on the other side.
    
    geom --- The geometry of the dual carriageway to modify
    other_geom --- The geometry of the other dual carriageway
    isU --- Specifies if this is the U side of the other geometry
    firstVertex --- Specifies if this is the first vertex of geom
    """
    other_left_p = get_vertex(other_geom.geoms[0], isU)
    other_right_p = get_vertex(other_geom.geoms[1], not isU)

    if isU == firstVertex:
        temp_p = other_left_p
        other_left_p = other_right_p
        other_right_p = temp_p

    left_cw = geom.geoms[0] # Get left carriage way
    right_cw = geom.geoms[1] # Get right carriage way
    
    other_left_cw = other_geom.geoms[1 if isU == firstVertex else 0]
    other_right_cw = other_geom.geoms[0 if isU == firstVertex else 1]
    
    theta = line_azimuth(left_cw, firstVertex, min_length=5)
    theta = theta - line_azimuth(other_left_cw, isU, min_length=5)
    theta = abs(theta)
    
    straight = 135 < theta < 225 or 315 < theta or theta < 45

    p = left_cw.intersection(other_left_cw)
    if (p.type == "Point" or p.type == "MultiPoint") and not p.is_empty:
        if p.type == "MultiPoint":
            p = pick_closests_to(p, Point(left_cw.coords[0 if firstVertex else -1]))
        proj = left_cw.project(p)
        if firstVertex:
            left_line = substring(left_cw, proj, left_cw.length)
        else:
            left_line = substring(left_cw, 0, proj)
    else:
        left_p = get_vertex(left_cw, firstVertex) # First point of left line
        new_left_p = midpoint(left_p, other_left_p)
        
        if firstVertex:
            left_line = LineString([new_left_p] + left_cw.coords[1 if straight else 0:])
        else:
            left_line = LineString(left_cw.coords[:-1 if straight else None] + [new_left_p])
    
    p = right_cw.intersection(other_right_cw)
    if (p.type == "Point" or p.type == "MultiPoint") and not p.is_empty:
        if p.type == "MultiPoint":
            p = pick_closests_to(p, Point(right_cw.coords[-1 if firstVertex else 0]))
        proj = right_cw.project(p)
        if firstVertex:
            right_line = substring(right_cw, 0, proj)
        else:
            right_line = substring(right_cw, proj, right_cw.length)
    else:
        right_p = get_vertex(right_cw, not firstVertex) # Last point of right line
        new_right_p = midpoint(right_p, other_right_p)
        if firstVertex:
            right_line = LineString(right_cw.coords[:-1] + [new_right_p])
        else:
            right_line = LineString([new_right_p] + right_cw.coords[1:])

    return MultiLineString([left_line, right_line])

def find_next_road(f, neighbours, nodeid, firstVertex):
    """Returns the feature that counts as a continuation of this road (if any)"""
    
    if len(neighbours) == 0:
        return None
    
    if len(neighbours) == 1:
        return neighbours.iloc[0]
    
    other_feature = []
    
    axis_geom = as_line(f['geometry'])
    a = get_line_azimuth(axis_geom, not firstVertex)
    
    # Find out if there is only one other road with the same number of carriageways
    similar = 0 # number of similar carriageway-type roads in this intersection
    cw_count = count_carriageways(f)
    for i in range(len(neighbours)):
        n = neighbours.iloc[i]
        if count_carriageways(n) == cw_count:
            similar = similar + 1
            other_feature = n
    
    # If there is only one similar and this is a three-way, pick the similar
    if similar == 1 and len(neighbours) == 2:
        return other_feature
    
    ccc = neighbours.apply(count_carriageways, axis=1) == (1 if cw_count == 2 else 2)
    force_same_cw = len(neighbours[ccc]) == 2
    
    # Angle tolerance for deciding a continuation
    angle_tol = 20
    
    # Special case for when all lines are dual carriageways
    tab = [len(neighbours.iloc[i]['carriage_ways'].geoms) == 2 for i in range(len(neighbours))]
    if all(tab) and len(f['carriage_ways'].geoms) == 2:
        other_feature = []
        angle_tol = 45
    
    for i in range(len(neighbours)):
        n = neighbours.iloc[i]
        n_a = get_line_azimuth(as_line(n['geometry']), n['u'] != nodeid)
        if abs(n_a - a) < 180 + angle_tol and abs(n_a - a) > 180 - angle_tol:
            if cw_count == 1 and force_same_cw and count_carriageways(n) != cw_count:
                continue
            other_feature = n
    
    if isinstance(other_feature, list):
        return None
    else:
        return other_feature

def join_side(geom, nodeid, f, neighbours, next_feature, firstVertex=True):
    """Returns a geometry that has joined with the adjacent geometry of one side.
    
    geom --- The geometry to alter.
    nodeid --- The node id around which the join will occur.
    f --- The input feature.
    neighbours --- The list of neighbouring features in this intersection
    next_feature --- The feature classified as a "continuation" of this one
    firstVertex --- True if the first vertex of the geometry will be altered, or False if it's the last one.
    """
    if len(neighbours) == 0:
        return geom
    
    if all(count_carriageways(neighbours.iloc[i]) == 1 for i in range(len(neighbours))):
        # Pick first feature, because they are all single carriageways and they meet at the same point
        other_feature = neighbours.iloc[0]
        
        p = get_vertex(other_feature['geometry'], other_feature['u'] == nodeid)

        return merge_dual_to_single(geom, p, firstVertex)
    
    if not next_feature is None:
        if count_carriageways(next_feature) == 1:
            p = get_vertex(next_feature['geometry'], next_feature['u'] == nodeid)
            return merge_dual_to_single(geom, p, firstVertex)
        else:
            return join_dual_ways(geom, next_feature['carriage_ways'], next_feature['u'] == nodeid, firstVertex)
    
    return geom

def get_uuid(f):
    """Returns the uuid of a feature or np.nan"""
    
    if f is None:
        return np.nan
    else:
        return f['uuid']

def join_carriageways(f):
    """Returns the carriageways processed so that they are joined with their neighbours"""
    
    # Escape in the degenerate case of a 3-parts geometry (yes, it can happen)
    # Ideally, we should raise an exception here, 
    if len(f['carriage_ways'].geoms) == 3:
        warnings.warn("Feature {} has {} parts! We are recovering its original geometry.".format(f['uuid'], len(f['carriage_ways'].geoms)))
        return [f['geometry'], np.nan, np.nan]
    
    # Identify degenerate case where this is a short segment (<1m) and
    # is adjacent to single cariageways, so we collapse it.
    u_neighbours = find_neighbouring_edges(f, f['u'])
    v_neighbours = find_neighbouring_edges(f, f['v'])
    
    if f['u'] == f['v']:
        u_next = f
        v_next = f
    else:
        u_next = find_next_road(f, u_neighbours, f['u'], True)
        v_next = find_next_road(f, v_neighbours, f['v'], False)
    
    if count_carriageways(f) == 1:
        return [f['carriage_ways'], get_uuid(u_next), get_uuid(v_next)]

    if f['u'] == f['v']:
        geom = join_dual_ways(f['carriage_ways'], f['carriage_ways'], True, False)
        geom = join_dual_ways(geom, f['carriage_ways'], False, True)
        return [geom, get_uuid(f), get_uuid(f)]
    
    if f['length'] < 1:
        if len(u_neighbours) == 0 or len(v_neighbours) == 0:
            return [f['geometry'], np.nan, np.nan]

        if len(u_neighbours) == 1 and len(v_neighbours) == 1 and count_carriageways(u_neighbours.iloc[0]) == 1 and count_carriageways(v_neighbours.iloc[0]) == 1:
            return [f['geometry'], get_uuid(u_next), get_uuid(v_next)]

    try:
        geom = join_side(f['carriage_ways'], f['u'], f, u_neighbours, u_next, True)
        geom = join_side(geom, f['v'], f, v_neighbours, v_next, False)
    except Exception as e:
        print(f['uuid'])
        raise e
    
    uuid_u = get_uuid(u_next)
    uuid_v = get_uuid(v_next)
    
    return [geom, uuid_u, uuid_v]

tmp = roads_width_final_final.apply(join_carriageways, axis=1)
roads_width_final_final['fixed_carriageways'] = [a[0] for a in tmp]
roads_width_final_final['u_next'] = [a[1] for a in tmp]
roads_width_final_final['v_next'] = [a[2] for a in tmp]

geopandas.GeoDataFrame(roads_width_final_final[['uuid', 'u', 'v', 'u_next', 'v_next', 'osmid', 'name', 'oneway', 'fixed_carriageways']], geometry='fixed_carriageways', crs='EPSG:28992').to_file('output/fixed_carriageways.geojson', driver='GeoJSON')

print("Done")

# 2084407287
# 4034800732
# f = roads_width_final_final[roads_width_final_final['uuid'] == 3852453425].iloc[0]
# join_carriageways(f)[2]

/Users/liberostelios/Dropbox/TU Delft/PhD Thesis/22 - Roads dataset/src/venv/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: Feature 469375467 has 3 parts! We are recovering its original geometry.
/Users/liberostelios/Dropbox/TU Delft/PhD Thesis/22 - Roads dataset/src/venv/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: Feature 2020816938 has 3 parts! We are recovering its original geometry.


Done


**WARNING: (from Breda dataset)** 2495911722 should have a `v_next` because it's opposite road assigned it as it's next feature.

## Split carriageways to features

Carriageways should be now converted to individual features:

In [232]:
from shapely.geometry import LineString
import uuid
from cityhash import CityHash32

def create_uuid(f):
    return CityHash32(uuid.uuid4().hex)

def reverse_if_dual(f):
    if f['dual']:
        return LineString(f['geom'].coords[::-1])
    
    return f['geom']

carriageways = roads_width_final_final.copy().rename(columns={
                                                             'geometry': 'axis_geom',
                                                             'fixed_carriageways': 'geom',
                                                             'uuid': 'road_uuid'
                                                             })

carriageways.drop('carriage_ways', 1, inplace=True)
carriageways.drop('gid', 1, inplace=True)

carriageways = geopandas.GeoDataFrame(carriageways, geometry="geom", crs="EPSG:28992")

# Compute "dual" carriageway attribute
carriageways['dual'] = carriageways.apply(lambda f: len(f['geom'].geoms) > 1, axis=1)

# Explode the MultiLineStrings to LineStrings (so duals will become two features)
carriageways = carriageways.explode()

# Make the exploding index a value to define the side of the carriageway: 0 - left, 1 - right
carriageways = carriageways.reset_index(level=-1).rename(columns={'level_1': 'side'})

# Reverse geometries of ex-dual-carriageways
carriageways['geom'] = carriageways.apply(reverse_if_dual, axis=1)

# Create UUIDs
carriageways['uuid'] = carriageways.apply(create_uuid, axis=1)

# Make sure no collisions exist between UUIDs
assert len(pd.unique(carriageways['uuid'])) == len(carriageways['uuid'])

carriageways.drop(['axis_geom', 'geometry_right'], 1).to_file('output/carriageways.geojson', driver='GeoJSON')

In [233]:
len(carriageways[carriageways["dual"] == True]) / 2

2812.0

## Fix intersections

Given the continuous carriageways, we need to deal with intersections now.

### Trim and split lines at intersections

We need to make sure that lines are splitted (if they "continue" across the interesction) or get trimmed (if that's their end). We determine contiuation or termination by the existence or absense (respectively) of a `u_next`/`v_next` value.

At the end of this step, we have all carriageways as individual `MultiLineStrings`, which contain one or more `LineStrings` that are topologically valid. So, later on we'll have to explode those `MultiLineStrings` to get the individual carriageway segments.

In [234]:
from shapely.ops import substring, split

def get_feature(df, col_name, value):
    return df[df[col_name] == value].iloc[0]

def find_neighbouring_features(G, nodeid, df, uuid_col='uuid', exclude=[], geom_col='geom', min_length=3.5):
    """Returns the road UUIDs of the neighbours of a feature at a given node
    
    G --- The network to lookup
    nodeid --- The id of the node for which to look for neighbours
    df --- The dataframe to find features
    uuid_col --- The name of the column that contains the edge's uuid
    exclude --- A list of uuids to exclude from the returning list
    geom_col --- The geometry column to check for min_length
    min_length --- If a road segment is shorter than this, then also "bring" the next one
    """
    
    uuids = [a['uuid'] for u, v, a in G.in_edges(nodeid, data=True)] + [a['uuid'] for u, v, a in G.out_edges(nodeid, data=True)]
    
    if isinstance(exclude, list):
        for i in exclude:
            if i in uuids:
                uuids.remove(i)
    else:
        if exclude in uuids:
            uuids.remove(exclude)
    
    new_uuids = []
    for uuid in uuids:
        res = df[df[uuid_col] == uuid]
        if len(res) > 0:
            f = res.iloc[0]
            if f[geom_col].length < min_length:
                if not np.isnan(f['u_next']):
                    new_uuids.append(int(f['u_next']))
                if not np.isnan(f['v_next']):
                    new_uuids.append(int(f['v_next']))
    
    uuids = uuids + new_uuids
    uuids = list(set(uuids))
    
    if isinstance(exclude, list):
        for i in exclude:
            if i in uuids:
                uuids.remove(i)
    else:
        if exclude in uuids:
            uuids.remove(exclude)
    
    return df[df[uuid_col].isin(uuids)]

def move_end_vertex(line, p, firstVertex):
    """Move the first or last vertex of a line to the given point (p)."""
    
    if firstVertex:
        coords = [p] + line.coords[1:]
    else:
        coords = line.coords[:-1] + [p]
    
    return LineString(coords)

def is_dual_merge(lines, nodeid):
    """Returns True if the two carriageways of road_uuid meet at nodeid
    
    lines --- A dataframe that should contain both carriageways
    nodeid --- The node to check against
    """
    
    left = lines[lines['side'] == 0].iloc[0] # TODO: Add an assert here
    right = lines[lines['side'] == 1].iloc[0] # TODO: Add an assert here
    
    if left['u'] == left['v']:
        return False
    
    p_left = left['geom'].boundary[1 if left['u'] == nodeid else 0]
    p_right = right['geom'].boundary[0 if right['u'] == nodeid else 1]
    
    return p_left.equals(p_right)
    

def trim_at_intersection(geom, f, nodeid, force_split=False):
    neighbours = find_neighbouring_features(network_graph,
                                            nodeid,
                                            df=carriageways,
                                            uuid_col='road_uuid',
                                            exclude=f['road_uuid'])
    
    # This marks the special case where this segment is dealt as the "not last"
    # because the next one is too short and we are dealing with it.
    not_last_segment = False
    
    # This is to bypass a special case where the next segment is too small and this segment
    # might eventually be crossing with the next segment's perpendicular dual carriageway road.
    if f["dual"] == False:
        next_id = f["u_next" if f['u'] == nodeid else "v_next"]
        if not np.isnan(next_id):
            next_road = carriageways[carriageways['road_uuid'] == int(next_id)].iloc[0]
            if next_road['geom'].length < 3:
                next_neighbours = find_neighbouring_features(network_graph,
                                                        next_road['v' if nodeid == next_road['u'] else 'u'],
                                                        df=carriageways,
                                                        uuid_col='road_uuid',
                                                        exclude=[f['road_uuid'], next_road['road_uuid']])
                neighbours = pd.concat([neighbours, next_neighbours])
                                                
                force_split = not np.isnan(next_road["v_next" if next_road['u'] == nodeid else "u_next"])
                
                not_last_segment = True
    
    # Find the crossing point
    all_lines = shapely.ops.unary_union(neighbours.reset_index()['geom'])
    cross_p = all_lines.intersection(geom)
    
    if cross_p.is_empty:
        if geom.type == "LineString":
            return MultiLineString([geom])
        else:
            return geom
    
    # This is something to fix a bizarre issue with shapely during the next step
    # Check https://github.com/Toblerity/Shapely/issues/952
    geom = shapely.wkt.loads(geom.wkt)
    all_lines = shapely.wkt.loads(all_lines.wkt)

    # Split the line according to the other lines
    parts = list(split(geom, all_lines).geoms)
    
    for p in parts:
        if p.length < 0.001:
            parts.remove(p)
    
    # Case where we just split
    if len(parts) == 1:
        return MultiLineString(parts)
    
    if f['dual'] == False:
        if f['u'] == nodeid:
            end_p = f['geom'].boundary[0]
        else:
            end_p = f['geom'].boundary[1]
        
        dest_p = None
        is_merge = False
        for i, n in neighbours.set_index('uuid').iterrows():
            if n['dual'] == True and n['geom'].intersects(geom):
                pp = n['geom'].intersection(geom)
                pp = pp.difference(geom.boundary)
                # TODO: We need to exclude cases where this dual carriageway meets its other part on this side
                if pp.is_empty:
                    continue
                
                is_start = (n['u'] == nodeid) == (n['side'] == 1)

                # Check if this is a merging dual carriageway
                if is_dual_merge(neighbours[neighbours['road_uuid'] == n['road_uuid']], nodeid):
                    dest_p = pp
                else:
                    dest_p = Point(n['geom'].coords[0 if is_start else -1])
                
                break
        
        if cross_p.type == "MultiPoint" and not dest_p is None and not_last_segment == False:
            if is_merge:
                if f['u'] == nodeid:
                    parts = parts[1:]
                else:
                    parts = parts[:-1]
            else:
                if f['u'] == nodeid:
                    first_part = parts[0]
                    first_part = move_end_vertex(first_part, dest_p, False)

                    second_part = parts[1]
                    second_part = move_end_vertex(second_part, dest_p, True)

                    rest = parts[2:]

                    parts = [first_part, second_part] + rest
                else:
                    main_part = parts[:-2]

                    second_to_last_part = parts[-2]
                    second_to_last_part = move_end_vertex(second_to_last_part, dest_p, False)

                    last_part = parts[-1]
                    last_part = move_end_vertex(last_part, dest_p, True)

                    parts = main_part + [second_to_last_part, last_part]

                    
    # Cases where we might split and move the vertices or create a twoway segment
    if force_split:
        # We need to check some things for single carriageways
        next_uuid = f['u_next'] if f['u'] == nodeid else f['v_next']
        next_f = neighbours[neighbours['road_uuid'] == next_uuid].iloc[0]
        
        # Special case where we need to duplicate the last part of the linestring (to make a twoway)
        if is_one_way(f) and len(neighbours.groupby('road_uuid')) == 3:
            if next_f['dual'] == True:
                if f['u'] == nodeid:
                    parts = [reverse_line(parts[0])] + parts
                else:
                    parts = parts + [reverse_line(parts[-1])]
        
        # Special TriAn(na)gle
        next_segment = carriageways[carriageways['road_uuid'] == next_uuid].iloc[0]
        if cross_p.type == "MultiPoint" and f['dual'] == True and next_segment["dual"] == False:
            if (f['u'] == nodeid) == (f['side'] == 0):
                return MultiLineString(parts[:-1])
            else:
                return MultiLineString(parts[1:])
        
        return MultiLineString(parts)
    
    if cross_p.type == "MultiPoint" and (f['geom'].boundary[0] in cross_p.geoms or f['geom'].boundary[1] in cross_p.geoms) and not not_last_segment:
        return MultiLineString(parts)
    
    if cross_p in f['geom'].boundary:
        return MultiLineString(parts)
    
    if f['u'] == nodeid:
        if f['dual'] == True and f['side'] == 0:
            return MultiLineString(parts[:-1])
        else:
            return MultiLineString(parts[1:])
    else:
        if f['dual'] == True and f['side'] == 0:
            return MultiLineString(parts[1:])
        else:
            return MultiLineString(parts[:-1])

def trim_feature(f):
    if f['geom'].coords[0] == f['geom'].coords[-1]:
        return MultiLineString([f['geom']])
    
    try:
        geom = trim_at_intersection(f['geom'], f, f['u'], not np.isnan(f['u_next']))
        geom = trim_at_intersection(geom, f, f['v'], not np.isnan(f['v_next']))
        
        return shapely.wkt.loads(geom.wkt)
    except Exception as e:
        print(f["uuid"])
        raise e

carriageways['trimmed'] = carriageways.apply(trim_feature, axis=1)

geopandas.GeoDataFrame(carriageways.drop(['axis_geom', 'geometry_right', 'geom'], 1),
                       geometry="trimmed",
                       crs="EPSG:28992").to_file('output/carriageways_trimmed.geojson', driver='GeoJSON')

print("Done")

# f = carriageways[carriageways['uuid'] == 1586322538].iloc[0]
# trim_feature(f)

Done


### Extend and fix complex intersections

If a carriageway meets a dual road that is continuous, then we need to do certain things:
- In case of no continuation of the carriageway, we need to extend to the end node of the opposite side
- In case of continuation where this is dual and the next road segment is single, we need to make a triangle.

**TODO:** Deal with degenerate case of sideroads. Definition: an intersection of 4 roads, with two being dual and two being single. If the dual are a continuation of ~180° and the two single are <60° we have to move them etc.

In [235]:
def extend_at_intersection(geom, f, nodeid):
    neighbours = find_neighbouring_features(network_graph,
                                            nodeid,
                                            df=carriageways,
                                            uuid_col='road_uuid',
                                            exclude=f['road_uuid'])

    new_lines = list(geom.geoms)
    
    # This is to bypass a special case where the next segment is too small and this segment
    # might eventually be crossing with the next segment's perpendicular dual carriageway road.
    if f["dual"] == False:
        next_id = f["u_next" if f['u'] == nodeid else "v_next"]
        if not np.isnan(next_id):
            next_road = carriageways[carriageways['road_uuid'] == int(next_id)].iloc[0]
            if next_road['geom'].length < 3:
                next_neighbours = find_neighbouring_features(network_graph,
                                                        next_road['v' if nodeid == next_road['u'] else 'u'],
                                                        df=carriageways,
                                                        uuid_col='road_uuid',
                                                        exclude=[f['road_uuid'], next_road['road_uuid']])
                neighbours = pd.concat([neighbours, next_neighbours])
    
    next_uuid = f['u_next' if f['u'] == nodeid else 'v_next']
    for road_uuid, road in neighbours.groupby('road_uuid'):
        if len(road) == 1:
            if not np.isnan(next_uuid) and is_merge_with_next(f, next_uuid):
                cway = road.iloc[0]
                start_p = geom.intersection(cway['trimmed'])
                
                if start_p.type == "LineString":
                    continue
                
                end_p = Point(f['axis_geom'].geoms[0].coords[0 if f['u'] == nodeid else -1])
                
                if start_p == end_p:
                    continue
                
                dist = cway['trimmed'].project(start_p)
                
                if dist > 0 and dist < cway['trimmed'].length:                
                    # Connect the last part of the neighbour to this road
                    # We assume a single carriageway here
                    other_line = linemerge(cway['trimmed'])
                    if (cway['u'] == nodeid):
                        other_line = substring(other_line, 0, dist)
                    else:
                        other_line = substring(other_line, dist, other_line.length)

                    if (cway['u'] == nodeid) != ((f['u'] == nodeid) == (f['side'] == 1)):
                        other_line = LineString(list(other_line.coords[::-1]))

                    if (f['u'] == nodeid) == (f['side'] == 1):
                        geom = MultiLineString([other_line] + list(geom.geoms))
                    else:
                        geom = MultiLineString(list(geom.geoms) + [other_line])

                    return geom

                    new_lines = new_lines + [LineString((start_p, end_p))]
            
            continue
        
        for uuid, cway in road.set_index('uuid').iterrows():
            print(uuid)
            if cway['road_uuid'] != f['u_next'] and cway['road_uuid'] != f['v_next'] and f['trimmed'].intersects(cway['trimmed']):                
                side = int(not cway['side'])
                
                other_side = road[road['side'] == side].iloc[0]
                
                start_p = geom.intersection(cway['trimmed'])
                
                if start_p.type == "LineString":
                    continue
                
                if f['dual']:
                    from_start = (f['u'] == nodeid) == (f['side'] == 1)
                else:
                    from_start = f['u'] == nodeid
                
                # That's probably a line crossing the same line in both sides
                if start_p.type == "MultiPoint":
                    start_p = linemerge(f['trimmed']).boundary[0 if from_start else 1]
                
                if not np.isnan(next_uuid) and is_merge_with_next(f, next_uuid):
                    end_p = Point(f['axis_geom'].geoms[0].coords[0 if f['u'] == nodeid else -1])
                else:
                    # We'll use the "geom" because it's more reliable regarding the order of boundaries
                    if (other_side['u'] == nodeid) == (side == 1):
                        end_p = other_side['geom'].boundary[0]
                    else:
                        end_p = other_side['geom'].boundary[1]
                
                if end_p in geom.boundary:
                    continue
                
                if from_start:
                    end_p, start_p = (start_p, end_p)
                    
                if start_p == end_p:
                    continue

                new_lines = new_lines + [LineString((start_p, end_p))]
    
    geom = MultiLineString(new_lines)
    
    return geom

def next_too_short(f, nodeid):
    next_road = carriageways[carriageways['road_uuid'] == int(f['u_next' if f['u'] == nodeid else 'v_next'])].iloc[0]
    if next_road['geom'].length < 3:
        return np.isnan(next_road["v_next" if next_road['u'] == nodeid else "u_next"])
    
    return False

def is_merge_with_next(f, next_uuid):
    return f['dual'] == True and get_feature(carriageways, 'road_uuid', next_uuid)['dual'] == False

def extend_feature(f):
    geom = f['trimmed']
    
    if f['u'] == f['v']:
        return geom
    
    if f['u_next'] == f['v_next']:
        return geom
    
    try:
        if np.isnan(f['u_next']) or next_too_short(f, f['u']) or is_merge_with_next(f, f['u_next']):
            geom = extend_at_intersection(geom, f, f['u'])
        if np.isnan(f['v_next']) or next_too_short(f, f['v']) or is_merge_with_next(f, f['v_next']):
            geom = extend_at_intersection(geom, f, f['v'])
    except Exception as e:
        print(f["uuid"])
        raise e
    
    return geom

carriageways['extended'] = carriageways.apply(extend_feature, axis=1)

geopandas.GeoDataFrame(carriageways.drop(['axis_geom', 'geometry_right', 'geom', 'trimmed'], 1),
                       geometry="extended",
                       crs="EPSG:28992").to_file('output/carriageways_extended.geojson', driver='GeoJSON')

print("Done!")

# 2619280182
# 3591586436
# f = carriageways[carriageways['uuid'] == 3591586436].iloc[0]
# extend_feature(f)

571954163
2164942981
203009117
1565153019
4172662432
3184224663
1553171856
2448266783
2460821429
323913478
2113735273
272015292
4109436024
3427241712
2360374717
877051402
4145083961
3932889301
4086485951
908193179
3218142533
4032542782
647397741
2792507952
3279055159
2259363117
3248860205
3167304096
3094426106
2239863301
3905301604
2003254865
693746721
3931436282
2398826305
525475300
3081123195
900633026
2287757743
1773858621
3425567374
2632360335
2287757743
1773858621
3425567374
2632360335
3063829676
2095421663
4082589972
2509786001
3186543239
3335683598
3502110241
526049456
568197317
4065108666
248131590
3039970748
568197317
4065108666
248131590
3039970748
628177197
1895897975
628177197
1895897975
2527343144
1649884024
206116678
1211296676
2527343144
1649884024
206116678
1211296676
567005896
3606072140
1684348651
1156585012
796565322
2262821237
796565322
2262821237
3321814753
1586255886
3198753359
195290597
3571162016
298243502
3430815779
1013307190
3978760996
2611742190
3978760996
2

3783135009
34363081
3366725793
603953332
2187354659
4133861008
1939884662
2450870231
1832183080
4273192919
1849134551
2898054195
1832183080
4273192919
1849134551
2898054195
3739575983
1787138538
3522662342
852468287
1624252030
3978897437
1856370669
2043174311
3836497789
25803868
64759124
2042827055
3722957187
2536379073
811272648
820622271
3722957187
2536379073
811272648
820622271
773342732
2071525321
1772933785
304334208
1325531189
6884303
177438807
2514686298
1325531189
6884303
177438807
2514686298
2460821429
323913478
2113735273
272015292
4109436024
3427241712
224151908
2546277021
1103348389
4209251950
224151908
2546277021
1103348389
4209251950
404741081
3489491966
3889789208
160858152
404741081
3489491966
3889789208
160858152
2452233749
795344079
1082026478
3216385414
2452233749
795344079
1082026478
3216385414
1241250213
1621240154
1704032730
319869777
4091900127
2205932994
1912325466
3165653513
2462625870
3832182034
54420803
4022750800
3917727976
679046332
305860756
1147590870
391

third argument of GEOSProject_r must be Point*


3397891093
3993150710
1744175731
3348249259
3397891093
3993150710
1744175731
3348249259
2137153518
3294350910
3862692938
2625484049
2808364540
4178143353
3149236082
4261324315
1962442482
423159366
2742089795
1776606934
3115394331
1834074575
1364611318
105146672
1379247702
1688389480
3167037036
4250122886
1882679915
2313582559
4247469622
1594798118
69468703
3270305855
3855603042
3472090615
69468703
3270305855
3855603042
3472090615
2335775076
2903218968
489687180
767830273
1249421705
137927490
1831958502
700920339
646111054
1469772844
2335775076
2903218968
489687180
767830273
1249421705
137927490
1831958502
700920339
646111054
1469772844
4062843272
4152274113
2797960398
1929988689
239731402
3846513091
2367974619
2027108533
239731402
3846513091
2367974619
2027108533
3116857456
2318728602
124820138
2866577010
1047141945
3581425418
1181522300
1008358162
3173202616
3314850549
3173202616
3314850549
1638286637
3203774509
4007192091
2435113143
81988479
1478500810
681436491
784531675
2623180545


381090471
2004390219
2882698753
692615635
1397892076
380644105
3866190811
2843590408
2390056956
1115847395
1083707920
2918652323
204383310
1542337010
1230529828
2626893972
204383310
1542337010
1230529828
2626893972
3964479543
2231579764
4127045794
4036283243
1115063603
316646421
3694987592
779854368
3331897979
2687694707
1961474933
826976622
3331897979
2687694707
1961474933
826976622
3656696524
3217365958
4194900108
733359460
3656696524
3217365958
4194900108
733359460
2315257696
3306280896
2396783710
804292842
3132022394
739431354
490907488
3580487349
1131668245
1202293757
2919661852
1353577515
1725842038
3995192225
3337378622
3883596519
2155907007
1817809851
2200942784
3706516320
6478762
2248177824
961101836
2275096756
6478762
2248177824
961101836
2275096756
1938862806
2011595930
3856734151
3554558732
3063829676
2095421663
3069481596
1665998509
2099598087
214140625
3736699478
3025600241
3836926621
3553226832
37755575
4261753570
3516650819
2103843573
2166103076
2556113741
4058428907
24

2487399294
1375801031
2911856611
2028134085
544006820
2265049356
3842156703
1153377152
1230622803
2823660245
3115501534
851812883
1230622803
2823660245
3115501534
851812883
38274211
406470017
4107871378
2019943069
3818603821
1798779529
1893416031
1102914266
3818603821
1798779529
1893416031
1102914266
3329643328
1299512697
646111054
1469772844
3910002818
2968077848
971527825
1768809160
305107258
1675013117
857762254
2225290381
1151861961
3754877261
4114690377
319254438
749699363
979158930
2980373212
499833701
4281744914
1789732645
1684348651
1156585012
2162331521
3270733621
712561591
1260467232
4135244475
855900507
4006857460
3062525709
4135244475
855900507
4006857460
3062525709
2619646982
2884258906
2422000281
3752160714
3796458911
1811254562
449154257
3372111853
3191969186
558739128
1161791533
2476419868
1964754787
3309019656
809263313
2787817432
1255953057
1748578028
2473584572
3283755506
745713571
3553597975
3402949864
50964619
3655820362
3931826370
1314577146
3024937435
2813549908


1947942391
1595029359
3081658287
306820339
2464126546
2264398983
2976448002
750890699
3739575983
1787138538
4144664571
903008315
3739575983
1787138538
4144664571
903008315
1719674083
3331781155
1757824725
4208599016
1719674083
3331781155
1757824725
4208599016
1838379719
4267090292
1472370319
1127015024
2550743111
1085360319
3074320476
1273181637
3391520843
1365336008
3950855649
2634185646
3391520843
1365336008
3950855649
2634185646
1962442482
423159366
308911787
1796171221
3351207558
927367127
2106237387
2080324344
3978190784
3924344377
825941949
3049605883
1151861961
3754877261
1368041087
2042982751
1151861961
3754877261
1368041087
2042982751
2765493480
1551848173
1704830157
950551961
2440938703
2093774587
846424672
266034489
2440938703
2093774587
846424672
266034489
936937912
2610789603
4058428907
243810375
936937912
2610789603
4058428907
243810375
1468592463
1053289495
987277185
3656853117
1468592463
1053289495
987277185
3656853117
3767452482
1157097853
1762142265
734311492
41590794

1759956301
3412823448
3673280275
24039329
1135404142
666816161
4253526469
2403893498
1276364367
2592628081
3384215694
4022889458
2182331474
1556609535
4263789492
40025040
2074773001
288399768
3566126659
552283229
1516973962
373658412
1250161867
3660894965
91561839
2889041489
3779213526
145027428
91561839
2889041489
3779213526
145027428
27120304
647003837
2237801946
1970438053
27120304
647003837
2237801946
1970438053
4018472295
596506815
1932119388
1023645655
4018472295
596506815
1932119388
1023645655
2570318829
2940174872
453028652
3042216415
832661718
4221936711
1708358029
2177655901
832661718
4221936711
1708358029
2177655901
1784619470
1002997589
1665772358
1716324628
2672019286
597040483
2964907460
3895480707
1029686191
2005581205
1230664654
2327278752
819429118
1906106255
1142645757
1529992397
819429118
1906106255
1142645757
1529992397
2111694913
3704477368
4038150739
2490226632
3759545931
4074524555
2062857044
2149474920
3905870403
95607693
1592289676
123911150
3905870403
95607693

2635170077
869824129
840525481
242674502
2534714652
2808232252
4098077887
3358617284
2534714652
2808232252
4058136649
3405400216
2722724024
848673222
1959048644
2865080612
1993507488
1006246406
2231036092
1903912128
3497072866
1311351772
2048132972
341189533
432176538
1289906628
4148778205
1425278648
586362322
1927950189
4251137616
1108792835
4251137616
1108792835
2112487488
2084389692
2039275209
2930726465
1239945704
634646180
530240019
270717057
3774884419
3776835022
1849674651
3135402719
3327915394
3522582337
2904968262
4143597756
2424439475
3940156180
2020851760
4267085854
3444171879
2470531546
1428892630
3568510338
768391053
648826674
3027426565
1157973431
2430742419
600969382
1255953057
1748578028
3730222953
2000696966
404741081
3489491966
1941206016
1525227950
2962275462
3555858340
459952060
994785020
3755047651
1367400092
1034018904
1159804802
2182196901
887961617
2021617262
1496862186
1161867943
3704507560
214544323
3539166903
1561024814
744879137
4148330882
3444660941
2094014

### Fix intersections when dual carriageways merge to one

When a carriageway ends up in an intersection and merges to one, then we should try to "snap" it to the perpendicular roads (if any).

The main idea is that we get the second-to-last point of the carriage way (that is, the point of the base of the small "triangle") and we compute the projection to all neighbours of the intersection. We should pick one of them based on a rule (e.g. the one with the closest distance?) and then make a new segment there.

In [236]:
from shapely.ops import substring, split

def move_vertex(line, orig_c, dest_c):
    """Will move the vertex orig_c to the position of dest_c.
    
    line --- A LineString of MultiLineString to process
    orig_c --- The original coords
    dest_c --- The destination coords
    """
    
    if line.type == "LineString":
        return LineString([dest_c if c==orig_c else c for c in line.coords])
    else:
        return MultiLineString([move_vertex(l, orig_c, dest_c) for l in line.geoms])

def cut_line(line, p, reverse = False):
    """Cuts the (Multi)LineString at the given point p
    
    line    --- A LineString or a MultiLineString to process
    p       --- The point where to cut the line string
    reverse --- Cut from the beginning or the end
    """
    
    if line.type == "LineString":
        dist = line.project(p)
        if reverse:
            return substring(line, 0, dist)
        else:
            return substring(line, dist, line.length)
    
    # Case of MultiLineString
    one_line = linemerge(line)
    
    assert one_line.type == "LineString"
    
    geoms = []
    for l in line.geoms:
        if l.project(p) > 0 and l.project(p) < l.length:
            geoms.append(cut_line(l, p, reverse))
        else:
            dist = one_line.project(p)
            dist_start = one_line.project(l.boundary[0])
            dist_end = one_line.project(l.boundary[1])
            
            if not reverse and dist_start <= dist and dist_end <= dist:
                continue
            elif reverse and dist_start >= dist and dist_end >= dist:
                continue
            geoms.append(l)
    
    return MultiLineString(geoms)

def fix_precision(geom, decimals=3):
    if geom.type == "LineString":
        return LineString([np.round(p, decimals) for p in geom.coords])
    elif geom.type == "MultiLineString":
        lines = []
        for l in geom.geoms:
            lines.append(fix_precision(l))
        return MultiLineString(lines)

def fix_merged_at_intersection(geom, f, nodeid):
    neighbours = find_neighbouring_features(network_graph,
                                            nodeid,
                                            df=carriageways,
                                            uuid_col='road_uuid',
                                            exclude=f['road_uuid'])
    
    if len(neighbours.groupby('road_uuid')) < 2:
        return geom
    
    all_lines = shapely.ops.unary_union(neighbours.reset_index()['extended'])
    all_lines = fix_precision(all_lines)
    
    geom = fix_precision(geom)
    
    inter = all_lines.intersection(geom)
    if inter.type == "LineString" or inter.type == "MultiLineString":
        return geom
    
    # Here there is a MultiLineString so we need to pick the right one
    # and select the second-to-last point.    
    one_line = linemerge(geom)
    
    if one_line.type != "LineString":
        warnings.warn("Feature {} shares multiple lines with neighbours! Skipping the snapping.".format(f['uuid']))
        return geom
        
    assert one_line.type == "LineString"
    
    p = Point(one_line.coords[1 if (f['u'] == nodeid) == (f['side'] == 1) else -2])
    end_p = Point(one_line.coords[0 if (f['u'] == nodeid) == (f['side'] == 1) else -1])
    
    az = get_line_azimuth(f['axis_geom'].geoms[0], f['u'] != nodeid)
    
    orig_geom = geom
    
    for uuid, n in neighbours.set_index('uuid').iterrows():
        if n['dual'] == True:
            continue

        cross_p = n["extended"].intersection(geom)
        if cross_p.type == "LineString" and not cross_p.is_empty:
            continue
                
        n_az = get_line_azimuth(n['axis_geom'].geoms[0], n['u'] != nodeid)
        
        angle = abs(n_az - az)
        
        if (angle < 70) and cross_p in one_line.boundary:
            continue
        
        if angle < 120 or (angle > 240 and angle < 300):
            l = 0
            m = n
            while l < LineString([p, end_p]).length:
                l = l + m['trimmed'].length
                dist = m['trimmed'].project(p)
                if dist > 0.001 and dist < m['trimmed'].length - 0.001:
                    geom = orig_geom
                    proj_p = m['trimmed'].interpolate(dist)
                    
                    # We snap the "remaining" part of the road to the neighbour
                    geom = move_vertex(geom, p.coords[0], proj_p.coords[0])
                    geom = cut_line(geom, proj_p, (f['u'] == nodeid) == (f['side'] == 0))
                                        
                    # Connect the last part of the neighbour to this road
                    # We assume a single carriageway here
                    other_line = linemerge(m['trimmed'])
                    fix_precision(other_line)
                    if (n['u'] == nodeid):
                        other_line = substring(other_line, 0, dist)
                    else:
                        other_line = substring(other_line, dist, other_line.length)
                    
                    if (n['u'] == nodeid) != ((f['u'] == nodeid) == (f['side'] == 1)):
                        other_line = LineString(list(other_line.coords[::-1]))
                    
                    if (f['u'] == nodeid) == (f['side'] == 1):
                        geom = MultiLineString([other_line] + list(geom.geoms))
                    else:
                        geom = MultiLineString(list(geom.geoms) + [other_line])
                
                next_uuid = m['v_next' if m['u'] == nodeid else 'u_next']
                if np.isnan(next_uuid):
                    break
                else:
                    m = get_feature(carriageways, 'road_uuid', next_uuid)
    
    return geom

def fix_merged_feature(f):
    """Returns a carriageway 'snapped' to the closest line."""
    
    geom = f['extended']
        
    if f["dual"] == False:
        return geom
    
    lines = carriageways[carriageways['road_uuid'] == f['road_uuid']]
    
    try:
        if is_dual_merge(lines, f['u']):
            geom = fix_merged_at_intersection(geom, f, f['u'])

        if is_dual_merge(lines, f['v']):
            geom = fix_merged_at_intersection(geom, f, f['v'])
    except Exception as e:
        print(f['uuid'])
        raise e
    
    return geom

carriageways['snapped'] = carriageways.apply(fix_merged_feature, axis=1)

geopandas.GeoDataFrame(carriageways.drop(['axis_geom', 'geometry_right', 'geom', 'trimmed', 'extended'], 1),
                       geometry="snapped",
                       crs="EPSG:28992").to_file('output/carriageways_snapped.geojson', driver='GeoJSON')

print("Done!")

# f = carriageways[carriageways['uuid'] == 1171340426].iloc[0]
# fix_merged_feature(f)

Done!


# Cleanup

This is where we clean up the network.

In [293]:
all_uuids = []

def collect_uuid(f):
    all_uuids.append(f['uuid'])

carriageways.apply(collect_uuid, axis=1)

len(all_uuids)

31243

In [291]:
final_edges = geopandas.GeoDataFrame(carriageways.drop(['axis_geom', 'geometry_right', 'geom', 'trimmed', 'extended'], 1), geometry='snapped', crs='EPSG:28992')
final_edges.rename_geometry('geometry', inplace=True)

In [260]:
G = ox.graph_from_gdfs(nodes, final_edges)

In [277]:
nx.number_weakly_connected_components(G)

14869

In [280]:
G_input = ox.graph_from_gdfs(nodes, road_edges)
nx.number_weakly_connected_components(G_input)

print("Nodes before: {}".format(len(G_input.nodes)))
G_input = ox.utils_graph.remove_isolated_nodes(G_input)
print("Nodes after: {}".format(len(G_input.nodes)))

nx.number_weakly_connected_components(G_input)

14869

In [292]:
print("Nodes before: {}".format(len(G.nodes)))
G = ox.utils_graph.remove_isolated_nodes(G)
print("Nodes after: {}".format(len(G.nodes)))

nx.number_weakly_connected_components(G)

Nodes before: 38949
Nodes after: 24131


51

In [275]:
len(network_graph.edges)

54841

In [264]:
def create_network(edges, nodes):
    """
    Creates a network from the given edges and nodes, removing duplicate parts.
    """
    
    

#### TODO

- In the current "snapped" pass we need make a last segment that takes the shape of the road that we "snap" to, instead of just making a simple line.
- We need to add a last pass where we intersect all lines of an intersection between them so that we find linear intersection in order to add a node in the lines that don't have one.

## Several notes and TODOS

1. **(DONE)** Smooth out the carriageways between them:
    - Normal case (two double-carriageways meet), we compute the mid point of the endpoints and join the lines.
    - When a double-carriageway meets a single one, we join with a triangle.
2. Find outliers that are too wide and are a dead-end.
3. Compute the new nodes of the network.

# Introduce flyovers

- What about the width calculation of flyovers? Is there an attribute in BGT about it? Can we compute the width by "isolating" the network and flyover polygon?

# Validation and evaluation

- Make sure we don't have those pedestrian area that are actually roads, but they are inconveniently classified in BGT as `voetpad`.
- We need to rectify our conditions for outliers regarding our decision of carriageway number from width.
- **INVESTIGATION REQUIRED**: If <1m segment is dual carriageway and neighbouring to both a single and a dual carriageway then we should make sure the methodology works.
- Ensure completeless of network (run a simple routing problem)
- Compute decorations like traffic lights
- Potentionally compute more interesting algorithms, e.g. what the best route about de-icing given the width of street (exploiting the semantic surfaces here).